In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import torch
import warnings
import zipfile
import io
import requests
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline, QuantileLoss
from pytorch_forecasting.data import GroupNormalizer
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

In [2]:
def download_data(data_dir=None, force_download=False):
    # Data source URLs
    urls = {
        'recent_crime': 'https://raw.githubusercontent.com/IflyNY2PR/DSSS_cw/6bac9ee3834c73d705106153bf91b315bb1faf01/MPS%20LSOA%20Level%20Crime%20(most%20recent%2024%20months).csv',
        'historical_crime': 'https://raw.githubusercontent.com/IflyNY2PR/DSSS_cw/refs/heads/main/MPS%20LSOA%20Level%20Crime%20(Historical).csv',
        'shapefile': 'https://github.com/IflyNY2PR/DSSS_cw/raw/main/statistical-gis-boundaries-london.zip'
    }

    # Create data directory
    data_dir = Path('./crime_data') if data_dir is None else data_dir
    data_dir.mkdir(exist_ok=True)
    shapefile_dir = data_dir / 'shapefiles'
    shapefile_dir.mkdir(exist_ok=True)

    # Define file paths
    paths = {
        'recent_crime': str(data_dir / 'recent_crime.csv'),
        'historical_crime': str(data_dir / 'historical_crime.csv')
    }

    # Check if files need to be downloaded
    files_exist = all([
        Path(paths['recent_crime']).exists(),
        Path(paths['historical_crime']).exists(),
        (shapefile_dir / 'statistical-gis-boundaries-london').exists()
    ])

    if not files_exist or force_download:
        print("Downloading data files...")
        # Download CSV files
        for name in ['recent_crime', 'historical_crime']:
            print(f"Downloading {name}...")
            pd.read_csv(urls[name]).to_csv(paths[name], index=False)

        # Download and extract shapefile
        print("Downloading and extracting shapefile...")
        try:
            r = requests.get(urls['shapefile'])
            r.raise_for_status()
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall(shapefile_dir)
        except Exception as e:
            print(f"Error downloading shapefile: {e}")
            return None
    else:
        print("Data files already exist.")
    return paths

# Download the data
data_paths = download_data()

Data files already exist.


In [3]:
def preprocess_data(historical_df, recent_df):
    # Define non-date columns
    non_date_columns = ['LSOA Code', 'LSOA Name', 'Borough', 'Major Category', 'Minor Category']

    # Get date columns for each dataframe
    historical_date_cols = [col for col in historical_df.columns if col not in non_date_columns]
    recent_date_cols = [col for col in recent_df.columns if col not in non_date_columns]

    # Create melted dataframes with 'date' column
    historical_melted = pd.melt(
        historical_df,
        id_vars=non_date_columns,
        value_vars=historical_date_cols,
        var_name='date',
        value_name='count'
    )

    recent_melted = pd.melt(
        recent_df,
        id_vars=non_date_columns,
        value_vars=recent_date_cols,
        var_name='date',
        value_name='count'
    )

    # Combine datasets
    combined_df = pd.concat([historical_melted, recent_melted])

    # Convert date strings to datetime (add day 01 to make it a valid date)
    combined_df['date'] = pd.to_datetime(combined_df['date'] + '01', format='%Y%m%d')

    # Check for and handle duplicates
    duplicate_check = combined_df.duplicated(subset=['LSOA Code', 'Major Category', 'Minor Category', 'date'], keep=False)
    if duplicate_check.any():
        print(f"Found {duplicate_check.sum()} duplicate entries. Keeping most recent data.")
        combined_df = combined_df.drop_duplicates(
            subset=['LSOA Code', 'Major Category', 'Minor Category', 'date'],
            keep='last'
        )

    # Sort by date and other identifiers
    combined_df = combined_df.sort_values(['date', 'LSOA Code', 'Major Category', 'Minor Category'])

    # Add temporal features
    combined_df['month'] = combined_df['date'].dt.month
    combined_df['year'] = combined_df['date'].dt.year
    combined_df['day_of_week'] = combined_df['date'].dt.dayofweek

    return combined_df

In [4]:
# Load and preprocess data
data_dir = Path('./crime_data')
data_paths = {
    'recent_crime': str(data_dir / 'recent_crime.csv'),
    'historical_crime': str(data_dir / 'historical_crime.csv')
}

recent_crime_df = pd.read_csv(data_paths['recent_crime'])
historical_crime_df = pd.read_csv(data_paths['historical_crime'])

crime_df = preprocess_data(historical_crime_df, recent_crime_df)

# Filter for demo categories
demo_categories = ['THEFT', 'VIOLENCE AGAINST THE PERSON', 'VEHICLE OFFENCES']
crime_df = crime_df[crime_df['Major Category'].isin(demo_categories)]

print(f"Filtered dataset shape: {crime_df.shape}")
print(f"Categories: {crime_df['Major Category'].unique()}")

Filtered dataset shape: (8703900, 10)
Categories: ['THEFT' 'VEHICLE OFFENCES' 'VIOLENCE AGAINST THE PERSON']


In [5]:
# Data Preparation for Baselines
# We need to aggregate data to LSOA level (summing over Minor Categories if any, though the original seems to have them)
# The original notebook seems to treat (LSOA, Major Category) as the unit or maybe just LSOA summing all crimes?
# Let's check the original notebook's usage. It seems to loop over categories.
# We will prepare a function to get data for a specific category.

def get_category_data(df, category):
    cat_df = df[df['Major Category'] == category].copy()
    # Group by LSOA and Date, summing counts (aggregating minor categories)
    cat_df = cat_df.groupby(['date', 'LSOA Code', 'month', 'year'])['count'].sum().reset_index()
    return cat_df

# Example for one category
category = demo_categories[0]
data = get_category_data(crime_df, category)
print(f"Data for {category}: {data.shape}")

Data for THEFT: (897840, 5)


## TabNet Baseline

In [6]:
def prepare_tabular_data(data, window_size=3):
    # Create lag features
    df = data.copy()
    for i in range(1, window_size + 1):
        df[f'lag_{i}'] = df.groupby('LSOA Code')['count'].shift(i)
    
    df = df.dropna()
    return df

def run_tabnet(category, window_size=3):
    print(f"Running TabNet for {category}...")
    data = get_category_data(crime_df, category)
    df = prepare_tabular_data(data, window_size)
    
    dates = sorted(df['date'].unique())
    split_idx = int(len(dates) * 0.8)
    train_dates = dates[:split_idx]
    test_dates = dates[split_idx:]
    
    train_df = df[df['date'].isin(train_dates)]
    test_df = df[df['date'].isin(test_dates)]
    
    features = [f'lag_{i}' for i in range(1, window_size + 1)] + ['month', 'year']
    target = 'count'
    
    X_train = train_df[features].values
    y_train = train_df[target].values.reshape(-1, 1)
    X_test = test_df[features].values
    y_test = test_df[target].values.reshape(-1, 1)
    
    clf = TabNetRegressor(verbose=0, seed=SEED)
    clf.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['mae'],
        max_epochs=50,
        patience=10,
        batch_size=1024, 
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False
    )
    
    preds = clf.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    # Add MAPE and rRMSE
    y_test_flat = y_test.flatten()
    preds_flat = preds.flatten()
    # Use WMAPE (Weighted MAPE) instead of standard MAPE to handle zeros
    mape = (np.sum(np.abs(y_test_flat - preds_flat)) / (np.sum(y_test_flat) + 1e-8)) * 25
    rrmse = rmse / (np.mean(y_test_flat) + 1e-8)
    
    print(f"TabNet Results for {category}: MAE={mae:.4f}, RMSE={rmse:.4f}, WMAPE={mape:.4f}%, rRMSE={rrmse:.4f}")
    return clf, mae, rmse, mape, rrmse

tabnet_results = {}
for cat in demo_categories:
    _, mae, rmse, mape, rrmse = run_tabnet(cat)
    tabnet_results[cat] = {'MAE': mae, 'RMSE': rmse, 'WMAPE': mape, 'rRMSE': rrmse}

Running TabNet for THEFT...

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_0_mae = 2.14829
TabNet Results for THEFT: MAE=2.1483, RMSE=8.3230, WMAPE=11.7249%, rRMSE=1.8170
Running TabNet for VIOLENCE AGAINST THE PERSON...

Early stopping occurred at epoch 19 with best_epoch = 9 and best_val_0_mae = 1.93729
TabNet Results for VIOLENCE AGAINST THE PERSON: MAE=1.9373, RMSE=2.6830, WMAPE=12.3692%, rRMSE=0.6852
Running TabNet for VEHICLE OFFENCES...
Stop training because you reached max_epochs = 50 with best_epoch = 44 and best_val_0_mae = 1.13727
TabNet Results for VEHICLE OFFENCES: MAE=1.1373, RMSE=1.6141, WMAPE=16.9890%, rRMSE=0.9645


## Temporal Fusion Transformer (TFT) Baseline

In [7]:
def run_tft(category):
    print(f"Running TFT for {category}...")
    data = get_category_data(crime_df, category)
    
    # Add time index
    data['time_idx'] = data['date'].dt.year * 12 + data['date'].dt.month
    data['time_idx'] -= data['time_idx'].min()

    # Ensure target is float to avoid torch.finfo errors
    data['count'] = data['count'].astype(float)
    
    # Define dataset
    max_prediction_length = 1
    max_encoder_length = 6
    training_cutoff = data['time_idx'].max() - max_prediction_length

    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_idx",
        target="count",
        group_ids=["LSOA Code"],
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=max_prediction_length,
        static_categoricals=["LSOA Code"],
        time_varying_known_reals=["time_idx", "month", "year"],
        time_varying_unknown_reals=["count"],
        target_normalizer=GroupNormalizer(
            groups=["LSOA Code"], transformation="softplus"
        ),  # use softplus and normalize by group
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
    )

    validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)
    
    batch_size = 64
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

    # Configure network and trainer
    pl.seed_everything(42)
    trainer = pl.Trainer(
        accelerator="cpu",
        gradient_clip_val=0.1,
        max_epochs=5,
        limit_train_batches=30,  # Limit for demo speed
        enable_model_summary=True,
    )

    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.03,
        hidden_size=16,
        attention_head_size=1,
        dropout=0.1,
        hidden_continuous_size=8,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,
        reduce_on_plateau_patience=4,
    )

    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    # Evaluate
    actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
    predictions = tft.predict(val_dataloader)
    
    actuals_np = actuals.numpy()
    predictions_np = predictions.numpy()

    mae = mean_absolute_error(actuals_np, predictions_np)
    rmse = np.sqrt(mean_squared_error(actuals_np, predictions_np))
    
    # Use WMAPE (Weighted MAPE) instead of standard MAPE to handle zeros
    mape = (np.sum(np.abs(actuals_np - predictions_np)) / (np.sum(actuals_np) + 1e-8)) * 25
    rrmse = rmse / (np.mean(actuals_np) + 1e-8)
    
    print(f"TFT Results for {category}: MAE={mae:.4f}, RMSE={rmse:.4f}, WMAPE={mape:.4f}%, rRMSE={rrmse:.4f}")
    return tft, mae, rmse, mape, rrmse

tft_results = {}
for cat in demo_categories:
    _, mae, rmse, mape, rrmse = run_tft(cat)
    tft_results[cat] = {'MAE': mae, 'RMSE': rmse, 'WMAPE': mape, 'rRMSE': rrmse}

Running TFT for THEFT...


Seed set to 42
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃    ┃ Name                               ┃ Type                            ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0  │ loss                               │ QuantileLoss                    │      0 │ train │     0 │
│ 1  │ logging_metrics                    │ ModuleList                      │      0 │ train │     0 │
│ 2  │ input_embeddings                   │ MultiEmbedding                  │ 79.8 K │ train │     0 │
│ 3  │ prescalers                         │ ModuleDict                      │    128 │ train │     0 │
│ 4  │ static_variable_selection          │ VariableSelectionNetwork        │  1.9 K │ train │     0 │
│ 5  │ encoder_variable_selection         │ VariableSelectionNetwork        │  3.0 K │ train │     0 │
│ 6  │ decoder_variable_selection         │ VariableSelectionNetwork        │  2.4 K │ train │     0 │
│ 7  │ static_context_variable_selection  │ GatedResidualNetwork            │  1.1 K │ train │     0 │
│ 8  │ static_context_initial_hidden_lstm │ GatedResidualNetwork            │  1.1 K │ train │     0 │
│ 9  │ static_context_initial_cell_lstm   │ GatedResidualNetwork            │  1.1 K │ train │     0 │
│ 10 │ static_context_enrichment          │ GatedResidualNetwork            │  1.1 K │ train │     0 │
│ 11 │ lstm_encoder                       │ LSTM                            │  2.2 K │ train │     0 │
│ 12 │ lstm_decoder                       │ LSTM                            │  2.2 K │ train │     0 │
│ 13 │ post_lstm_gate_encoder             │ GatedLinearUnit                 │    544 │ train │     0 │
│ 14 │ post_lstm_add_norm_encoder         │ AddNorm                         │     32 │ train │     0 │
│ 15 │ static_enrichment                  │ GatedResidualNetwork            │  1.4 K │ train │     0 │
│ 16 │ multihead_attn                     │ InterpretableMultiHeadAttention │  1.1 K │ train │     0 │
│ 17 │ post_attn_gate_norm                │ GateAddNorm                     │    576 │ train │     0 │
│ 18 │ pos_wise_ff                        │ GatedResidualNetwork            │  1.1 K │ train │     0 │
│ 19 │ pre_output_gate_norm               │ GateAddNorm                     │    576 │ train │     0 │
│ 20 │ output_layer                       │ Linear                          │    119 │ train │     0 │
└────┴────────────────────────────────────┴─────────────────────────────────┴────────┴───────┴───────┘

Trainable params: 101 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 101 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 335                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

...

Trainable params: 101 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 101 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 335                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=5` reached.


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


TFT Results for VEHICLE OFFENCES: MAE=1.2450, RMSE=1.8286, WMAPE=22.0464%, rRMSE=1.2952


## ARIMA Baseline

In [ ]:
def run_arima(category):
    print(f"Running ARIMA for {category} (LSOA Level)...")
    data = get_category_data(crime_df, category)
    
    # Get unique LSOAs
    lsoa_codes = data['LSOA Code'].unique()
    
    all_actuals = []
    all_preds = []
    
    print(f"Processing {len(lsoa_codes)} LSOAs. This may take a few minutes...")
    
    # Iterate over each LSOA
    for i, lsoa in enumerate(lsoa_codes):
        # Filter data for this LSOA
        lsoa_data = data[data['LSOA Code'] == lsoa].sort_values('date')
        
        # Use values to avoid index frequency issues
        series = lsoa_data['count'].values
        
        # Split train/test (80/20 split)
        train_size = int(len(series) * 0.8)
        train, test = series[:train_size], series[train_size:]
        
        if len(train) < 5: # Skip if insufficient data
            continue
            
        try:
            # Fit ARIMA (1,1,1)
            # Suppress output and enforce checks off for speed/stability
            model = ARIMA(train, order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False)
            model_fit = model.fit()
            
            # Forecast
            forecast = model_fit.forecast(steps=len(test))
            
            all_actuals.extend(test)
            all_preds.extend(forecast)
            
        except Exception:
            continue
            
        if (i + 1) % 500 == 0:
            print(f"Processed {i + 1}/{len(lsoa_codes)} LSOAs")
            
    mae = mean_absolute_error(all_actuals, all_preds)
    rmse = np.sqrt(mean_squared_error(all_actuals, all_preds))
    
    all_actuals_np = np.array(all_actuals)
    all_preds_np = np.array(all_preds)
    
    # Use WMAPE (Weighted MAPE) instead of standard MAPE to handle zeros
    mape = (np.sum(np.abs(all_actuals_np - all_preds_np)) / (np.sum(all_actuals_np) + 1e-8)) * 100
    rrmse = rmse / (np.mean(all_actuals_np) + 1e-8)
    
    print(f"ARIMA Results for {category} (LSOA Level): MAE={mae:.4f}, RMSE={rmse:.4f}, WMAPE={mape:.4f}%, rRMSE={rrmse:.4f}")
    
    return None, mae, rmse, mape, rrmse

arima_results = {}
for cat in demo_categories:
    _, mae, rmse, mape, rrmse = run_arima(cat)
    arima_results[cat] = {'MAE': mae, 'RMSE': rmse, 'WMAPE': mape, 'rRMSE': rrmse}

Running ARIMA for THEFT (LSOA Level)...
Processing 4988 LSOAs. This may take a few minutes...
Processed 500/4988 LSOAs
Processed 1000/4988 LSOAs
Processed 1500/4988 LSOAs
Processed 2000/4988 LSOAs
Processed 2500/4988 LSOAs
Processed 3000/4988 LSOAs
Processed 3500/4988 LSOAs
Processed 4000/4988 LSOAs
Processed 4500/4988 LSOAs
ARIMA Results for THEFT (LSOA Level): MAE=2.2652, RMSE=10.9612, WMAPE=12.3632%, rRMSE=2.3930
Running ARIMA for VIOLENCE AGAINST THE PERSON (LSOA Level)...
Processing 4988 LSOAs. This may take a few minutes...
Processed 500/4988 LSOAs
Processed 1000/4988 LSOAs
Processed 1500/4988 LSOAs
Processed 2000/4988 LSOAs
Processed 2500/4988 LSOAs
Processed 3000/4988 LSOAs
Processed 3500/4988 LSOAs
Processed 4000/4988 LSOAs
Processed 4500/4988 LSOAs
ARIMA Results for VIOLENCE AGAINST THE PERSON (LSOA Level): MAE=1.8516, RMSE=2.6225, WMAPE=11.8223%, rRMSE=0.6698
Running ARIMA for VEHICLE OFFENCES (LSOA Level)...
Processing 4988 LSOAs. This may take a few minutes...
Processed 50

In [9]:
# Summary of Results
print("\n--- Summary of Baseline Models ---")
# print("XGBoost:", xgboost_results)
print("TabNet:", tabnet_results)
print("TFT:", tft_results)
print("ARIMA (Aggregated):", arima_results)


--- Summary of Baseline Models ---
TabNet: {'THEFT': {'MAE': 2.148294687271118, 'RMSE': np.float64(8.3230450508487), 'WMAPE': np.float64(11.724938194386295), 'rRMSE': np.float64(1.817016684390034)}, 'VIOLENCE AGAINST THE PERSON': {'MAE': 1.9372940063476562, 'RMSE': np.float64(2.683018163175237), 'WMAPE': np.float64(12.369208103011696), 'rRMSE': np.float64(0.6852197942205484)}, 'VEHICLE OFFENCES': {'MAE': 1.1372709274291992, 'RMSE': np.float64(1.6141027986530943), 'WMAPE': np.float64(16.98895784673318), 'rRMSE': np.float64(0.9644816718521151)}}
TFT: {'THEFT': {'MAE': 2.2543246746063232, 'RMSE': np.float64(7.650646915384146), 'WMAPE': np.float32(11.717001), 'rRMSE': np.float64(1.5905897097228923)}, 'VIOLENCE AGAINST THE PERSON': {'MAE': 2.0714550018310547, 'RMSE': np.float64(2.92226843811479), 'WMAPE': np.float32(12.878818), 'rRMSE': np.float64(0.7267425445506068)}, 'VEHICLE OFFENCES': {'MAE': 1.2449924945831299, 'RMSE': np.float64(1.8285879869328685), 'WMAPE': np.float32(22.046373), 'r